**Test / Example note book of the functions executed in card_analyzer.py**

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load example dataset

In [21]:
import os
import sys
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [22]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
from utils import *
from graphs import *
from card_analyzer import *

In [15]:
# Set path of the folder containing dataset
dataset_FolderPath = Path.cwd().parent / 'data' # @dev TBC before each use

# Set path of the File
dataset_FileName = 'AllPrintings.json'
dataset_FilePath = dataset_FolderPath / dataset_FileName

In [16]:
# Load all datasets
data = pd.read_json(dataset_FilePath)
data = data.iloc[2:]['data'] # 2 first rows of JSON files are metadata

In [17]:
# Load a test dataset
set_code = 'OTJ'
cards = load_set(data, set_code, restriction='base_set')
cards

,name,keywords,manaValue,manaCost,colorIdentity,power,toughness,rarity,types,text
0,Another Round,NaN,3.0,{X}{X}{2}{W},[W],0.0,0.0,rare,[Sorcery],"Exile any number of creatures you control, the..."
1,Archangel of Tithes,[Flying],4.0,{1}{W}{W}{W},[W],3.0,5.0,mythic,[Creature],Flying\nAs long as Archangel of Tithes is unta...
2,Armored Armadillo,[Ward],1.0,{W},[W],0.0,4.0,common,[Creature],Ward {1} (Whenever this creature becomes the t...
3,Aven Interrupter,"[Flash, Flying, Plot]",3.0,{1}{W}{W},[W],2.0,2.0,rare,[Creature],"Flash\nFlying\nWhen Aven Interrupter enters, e..."
4,Bounding Felidar,[Saddle],6.0,{5}{W},[W],4.0,7.0,uncommon,[Creature],Whenever Bounding Felidar attacks while saddle...
...,...,...,...,...,...,...,...,...,...,...
266,Botanical Sanctum,NaN,0.0,NaN,"[G, U]",0.0,0.0,rare,[Land],Botanical Sanctum enters tapped unless you con...
267,Concealed Courtyard,NaN,0.0,NaN,"[B, W]",0.0,0.0,rare,[Land],Concealed Courtyard enters tapped unless you c...
268,Inspiring Vantage,NaN,0.0,NaN,"[R, W]",0.0,0.0,rare,[Land],Inspiring Vantage enters tapped unless you con...
269,Spirebluff Canal,NaN,0.0,NaN,"[R, U]",0.0,0.0,rare,[Land],Spirebluff Canal enters tapped unless you cont...


In [ ]:
card0 = Card(cards.iloc[0])
card0.show()

<bound method Card.get_features of Card(name                                                 Another Round
keywords                                                       NaN
manaValue                                                      3.0
manaCost                                              {X}{X}{2}{W}
colorIdentity                                                  [W]
power                                                          0.0
toughness                                                      0.0
rarity                                                        rare
types                                                    [Sorcery]
text             Exile any number of creatures you control, the...
Name: 0, dtype: object)>

In [8]:
# TEST 1: Apply filters

def isType(s, typelist):
    return any(t in s for t in typelist)

def isPermanent(s):
    return isType(s, ['Land', 'Creature', 'Artifact', 'Enchantment', 'Planeswalker', 'Battle'])

def createsToken(s):
    pattern = r'creat(e|es) .*? creature token'
    return bool(re.search(pattern, s, re.IGNORECASE | re.DOTALL))

def isETB(s):
    pattern = r'when .*? ente(r|rs)'
    return bool(re.search(pattern, s, re.IGNORECASE | re.DOTALL))

# Test
c = cards.loc[58]
c.text
"""
Outlaw Stitcher

"When Outlaw Stitcher enters, create a 2/2 blue and black Zombie Rogue creature token, then put two +1/+1 counters 
on that token for each spell you've cast this turn other than the first.\nPlot {4}{U} (You may pay {4}{U} and exile 
this card from your hand. Cast it as a sorcery on a later turn without paying its mana cost. Plot only as a sorcery.)"
"""

print(isType(c.types,['Creature']))
print(isPermanent(c.types))
print(createsToken(c.text))
print(isETB(c.text))

# all permanents that creates token on ETB
a = cards['types'].apply(isPermanent)
b = cards['text'].apply(createsToken)
c = cards['text'].apply(isETB)

result = cards[a & b & c]
result

True
True
True
True


,name,keywords,manaValue,manaCost,colorIdentity,power,toughness,rarity,types,text
17,Lassoed by the Law,NaN,4.0,{3}{W},[W],NaN,NaN,uncommon,[Enchantment],"When Lassoed by the Law enters, exile target n..."
24,Prosperity Tycoon,NaN,4.0,{3}{W},[W],4.0,2.0,uncommon,[Creature],"When Prosperity Tycoon enters, create a 1/1 re..."
58,Outlaw Stitcher,[Plot],4.0,{3}{U},[U],1.0,4.0,uncommon,[Creature],"When Outlaw Stitcher enters, create a 2/2 blue..."
98,Rakish Crew,NaN,3.0,{2}{B},[B],NaN,NaN,uncommon,[Enchantment],"When Rakish Crew enters, create a 1/1 red Merc..."
101,Rictus Robber,[Plot],4.0,{3}{B},[B],4.0,3.0,uncommon,[Creature],"When Rictus Robber enters, if a creature died ..."
127,Hellspur Posse Boss,NaN,4.0,{2}{R}{R},[R],2.0,4.0,rare,[Creature],Other outlaws you control have haste. (Assassi...
136,Prickly Pair,NaN,3.0,{2}{R},[R],2.0,2.0,common,[Creature],"When Prickly Pair enters, create a 1/1 red Mer..."
195,"Bonny Pall, Clearcutter",[Reach],6.0,{3}{G}{U}{U},"[G, U]",6.0,5.0,rare,[Creature],"Reach\nWhen Bonny Pall, Clearcutter enters, cr..."
202,"Ertha Jo, Frontier Mentor",NaN,4.0,{2}{R}{W},"[R, W]",2.0,4.0,uncommon,[Creature],"When Ertha Jo, Frontier Mentor enters, create ..."


In [103]:
# Test 2 : isBody function

def isBody(s):
    return (
        (
            isType(s['types'], ['Creature'])
        ) |
        (
            isPermanent(s['types'])
            & isETB(s['text'])
            & createsToken(s['text'])
        ) |
        (
            isType(s['types'], ['Instant', 'Sorcery'])
            & createsToken(s['text'])
        )
    )

# Test
c = cards.loc[9]
print(c.text)

isBody(c)

cards[cards[['types', 'text']].apply(isBody, axis=1)]

Destroy target tapped creature. You gain 2 life.


False

In [177]:
cards[cards[['types', 'text']].apply(isBody, axis=1)].head(20)

,name,keywords,manaValue,manaCost,colorIdentity,power,toughness,rarity,types,text
1,Archangel of Tithes,[Flying],4.0,{1}{W}{W}{W},[W],3.0,5.0,mythic,[Creature],Flying\nAs long as Archangel of Tithes is unta...
2,Armored Armadillo,[Ward],1.0,{W},[W],0.0,4.0,common,[Creature],Ward {1} (Whenever this creature becomes the t...
3,Aven Interrupter,"[Flash, Flying, Plot]",3.0,{1}{W}{W},[W],2.0,2.0,rare,[Creature],"Flash\nFlying\nWhen Aven Interrupter enters, e..."
4,Bounding Felidar,[Saddle],6.0,{5}{W},[W],4.0,7.0,uncommon,[Creature],Whenever Bounding Felidar attacks while saddle...
5,Bovine Intervention,NaN,2.0,{1}{W},[W],NaN,NaN,uncommon,[Instant],Destroy target artifact or creature. Its contr...
6,Bridled Bighorn,"[Saddle, Vigilance]",4.0,{3}{W},[W],3.0,4.0,common,[Creature],Vigilance\nWhenever Bridled Bighorn attacks wh...
7,Claim Jumper,[Vigilance],3.0,{2}{W},[W],3.0,3.0,rare,[Creature],"Vigilance\nWhen Claim Jumper enters, if an opp..."
8,Dust Animus,"[Flying, Plot]",2.0,{1}{W},[W],2.0,3.0,rare,[Creature],Flying\nIf you control five or more untapped l...
11,"Fortune, Loyal Steed","[Saddle, Scry]",3.0,{2}{W},[W],2.0,4.0,rare,[Creature],"When Fortune, Loyal Steed enters, scry 2.\nWhe..."
12,Frontier Seeker,NaN,2.0,{1}{W},[W],2.0,1.0,uncommon,[Creature],"When Frontier Seeker enters, look at the top f..."


In [185]:
# getBodyStats()

def isQuasiBody(card):
    return False

def getBodyStats(card):

    # pass the function it is not a body
    if not (isBody(card) | isQuasiBody(card)):
        return print("Not a body")

    # Filters
    ## 1. creature only
    filter1 = (
        isType(card['types'], ['Creature']) 
        & (not createsToken(card['text'])) 
    )
    ## 2. creature that creates a creature token on ETB
    filter2 = (
        isType(card['types'], ['Creature']) 
        & createsToken(card['text']) 
        & isETB(card['text'])
    ) 
    ## 3. permanent that creates a creature token on ETB
    filter3 = (
        isPermanent(card['types']) 
        & createsToken(card['text']) 
        & isETB(card['text'])
    ) 
    ## 4. instant / sorcery spell that creates a creature token
    filter4 = (
        isType(card['types'], ['Instant', 'Sorcery']) 
        & createsToken(card['text']) 
    ) 

    def findTokenPowerToughness(text):
        pattern = r'creat(e|es).*?(\b(?:\d+|X)/(?:\d+|X)\b).*?creature token'
        match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
        
        if match:
            # Extract the power/toughness string (e.g., "2/2" or "X/3")
            power, toughness = match.group(2).split('/')
            
            # Convert to integer if possible, or return 0 if 'X' is found
            power_int = int(power) if power.isdigit() else 0
            toughness_int = int(toughness) if toughness.isdigit() else 0
            
            return power_int, toughness_int
        else:
            return 0, 0  # Return (0, 0) if no match is found
    
    if filter1==True:
        bdType = 'Creature'
        bdManaValue = card['manaValue']
        bdPower = card['power']
        bdToughness = card['toughness']

    if filter2==True:
        bdType = 'Creature with ETB creature token'
        bdManaValue = card['manaValue']

        p, t = findTokenPowerToughness(card['text'])
        bdPower = card['power'] + p
        bdToughness = card['toughness'] + t

    if filter3==True:
        bdType = 'Non-creature with ETB creature token'
        bdManaValue = card['manaValue']
        
        p, t = findTokenPowerToughness(card['text'])
        bdPower = card['power'] + p
        bdToughness = card['toughness'] + t

    if filter4==True:
        bdType = 'Non-permanent with creature token'
        bdManaValue = card['manaValue']
        
        p, t = findTokenPowerToughness(card['text'])
        bdPower = p
        bdToughness = t

    return bdType, bdManaValue, bdPower, bdToughness

# Test
c = cards.loc[17]
print(c.text)

getBodyStats(c)

When Lassoed by the Law enters, exile target nonland permanent an opponent controls until Lassoed by the Law leaves the battlefield.
When Lassoed by the Law enters, create a 1/1 red Mercenary creature token with "{T}: Target creature you control gets +1/+0 until end of turn. Activate only as a sorcery."
yay
success


('Non-creature with ETB creature token', 4.0, nan, nan)